In [ ]:
import pandas as pd
from zipfile import ZipFile
import os
import re
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
words = ['word','word','word','word','word','word'] # enter your words to search here

In [ ]:
directory = 'All Data/Raw Journey Data/Until16May2020(no HQ media)/' #change directory here after unzipping journey data

import os
rawdata = {}
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(f'{directory}{filename}', 'r') as f:
            readjson = json.load(f)
            rawdata[readjson['id']]=readjson

In [ ]:
len(rawdata)

In [ ]:
datadf = pd.DataFrame(rawdata).transpose()

In [ ]:
datadf = datadf.drop(columns={'label','music_artist','music_title','preview_text','sentiment','type','folder'})
datadf.set_index('id', inplace=True)

In [ ]:
for i in range(len(datadf['date_journal'])):
    datadf['date_journal'][i] = dt.datetime.fromtimestamp((datadf['date_journal'][i])/1000).strftime("%Y-%m-%d %H:%M:%S")
    datadf['date_modified'][i] = dt.datetime.fromtimestamp((datadf['date_modified'][i])/1000).strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
datadf['no_punc'] = datadf['text']
for i in range(len(datadf['text'])):
    regex= '<.*?>'
    datadf['no_punc'][i] = re.sub(regex, '', (datadf['text'][i]))
    punct_re='[^\w\s]'
    datadf['no_punc'][i] = re.sub(punct_re, '', (datadf['no_punc'][i]))
    newline_re='\\n'
    datadf['no_punc'][i] = re.sub(newline_re, ' ', (datadf['no_punc'][i]))
    newline_re2='nbsp'
    datadf['no_punc'][i] = re.sub(newline_re2, ' ', (datadf['no_punc'][i]))
    datadf['no_punc'][i] = re.sub('\s+', ' ', datadf['no_punc'][i]).strip()

In [ ]:
datadf = datadf.sort_values(by='date_journal')

In [ ]:
datadf.head()

In [ ]:
datadf.to_csv(r'RawData.csv')

In [ ]:
tidy_format = datadf.no_punc.str.split(expand=True).stack().to_frame().reset_index(level=1)
tidy_format['num']=tidy_format['level_1']
tidy_format['word']=tidy_format[0]
tidy_format = tidy_format.drop(columns=[0,'level_1'])
tidy_format['lower'] =tidy_format['word'].str.lower()
tidy_format.to_csv('TidyFormat.csv')

In [ ]:
tidy_format

In [ ]:
word_counts = tidy_format['lower'].value_counts().to_frame().rename(columns={'lower':'count'})
word_counts.to_csv(r'WordCounts.csv')
word_counts

In [ ]:
timedata = pd.DataFrame(datadf[['date_journal']])
timedata['year'] = pd.DatetimeIndex(timedata['date_journal']).year
timedata['month'] = pd.DatetimeIndex(timedata['date_journal']).month
timedata['day'] = pd.DatetimeIndex(timedata['date_journal']).day
timedata['hour'] = pd.DatetimeIndex(timedata['date_journal']).hour
timedata['minute'] = pd.DatetimeIndex(timedata['date_journal']).minute
timedata['dayofweek'] = pd.DatetimeIndex(timedata['date_journal']).dayofweek
timedata = timedata.sort_values(by='date_journal')
timedata['day_num'] = (((pd.DataFrame(pd.DatetimeIndex(timedata.date_journal).date - 
                                      dt.datetime.strptime(timedata.date_journal[0], 
                                                           '%Y-%m-%d %H:%M:%S').date())[0])/np.timedelta64(1, 'D')).astype(int)).values
timedata['month_num'] = ((timedata['year'] - timedata['year'][0]) *12 ) + timedata['month'] - timedata['month'][0]
mapping_names = {1:'jan',2:'feb',3:'mar',4:'april',5:'may',6:'june',7:'july',8:'aug',9:'sept',10:'oct',11:'nov',12:'dec'}
timedata['month_name'] = timedata.date_journal.apply(lambda i:dt.datetime.strptime(i,'%Y-%m-%d %H:%M:%S').month).map(mapping_names)
timedata['year_month'] = timedata['year'].astype(str) + " " +timedata["month_name"]
timedata['minute_of_day'] = timedata['hour']*60 + timedata['minute']
timedata.to_csv('TimeData.csv')

In [ ]:
timedata.head()

In [ ]:
monthly = timedata.groupby('month').count()
plt.plot(monthly.index,monthly['year'])

In [ ]:
daily = timedata.groupby('day').count()
plt.plot(daily.index,daily['year'])

In [ ]:
week = timedata.groupby('dayofweek').count()
plt.plot(week.index,week['year'])

In [ ]:
hourly = timedata.groupby('hour').count()
plt.plot(hourly.index,hourly['year'])

In [ ]:
lex = pd.read_csv('All Data/SentimentAnalysisVaderLexicon.csv')
lex = lex.set_index('token')

polarities = tidy_format.merge(lex, left_on='word', right_on='token', copy=False, right_index=True)
polarities = polarities.groupby(polarities.index).sum().drop(columns='num')
pol_text = polarities.merge(datadf['no_punc'], left_on=polarities.index, right_on=datadf['no_punc'].index).rename(columns={'key_0':'id'})
pol_text = pol_text.merge(timedata['month'], left_on='id', right_on=timedata['month'].index)
pol_text.to_csv('Polarized Text.csv')
monthly_sent = pol_text.groupby('month').agg({'polarity':'sum'})
plt.plot(monthly_sent)

In [ ]:
monthly_sent_mean = pol_text.groupby('month').agg({'polarity':'mean'})
plt.plot(monthly_sent_mean)

In [ ]:
tidy_format['mentioned(entry)'] = tidy_format['lower'].isin(words).astype(int)
words_found = tidy_format.merge(timedata, on='id', how='left')[['mentioned(entry)','day_num','month_num', 'year_month']]
words_found_days = words_found.groupby('day_num').agg(func={'mentioned(entry)':'sum', 'month_num':'first', 'year_month':'first'})
words_found_days['mentioned(day)'] = (words_found_days['mentioned(entry)']!=0).astype(int)
words_found_days['avg_mentions(month)'] = words_found_days['mentioned(day)']
plt.plot(words_found_days.index, words_found_days['mentioned(day)'])

In [ ]:
words_found_months = words_found_days.groupby('year_month').agg(func={'mentioned(entry)':'sum', 'month_num':'first', 'year_month':'first', 'mentioned(day)':'sum', 'avg_mentions(month)':'mean'})
words_found_months = words_found_months.sort_values(by='month_num', ascending=True)

fig, ax = plt.subplots(figsize = (150,40))
ax.plot(words_found_months['mentioned(day)'])
plt.yticks(fontsize=70)
plt.xticks(fontsize=70, rotation=90)
plt.locator_params(axis='x', nbins=20)
plt.title(label='Number of days in a month the words were mentioned', fontsize=100)

In [ ]:
fig, ax = plt.subplots(figsize = (150,40))
ax.plot(words_found_months['avg_mentions(month)'])
plt.xticks(fontsize=70, rotation=90)
plt.yticks(fontsize=70)
plt.locator_params(axis='x', nbins=20)
plt.title(label='Proportion of days in a month when the words were mentioned', fontsize=100)

In [ ]:
def found (text, words):
    hits = 0
    for word in words:
        if word in str(text):
            hits = hits+1
    return(hits)

datadf['hits'] = datadf['no_punc'].apply(lambda x: found(x,words))

datadf['found'] = (datadf['hits']>0).astype(int)

datadf = datadf.merge(timedata, on='id', how='inner')[['no_punc','day_num','month_num','year_month','minute_of_day','found','hour']]

plt.figure(figsize=(100,10))
plt.xticks(fontsize=70, rotation=90)
plt.yticks(fontsize=70)
sns.distplot(datadf['minute_of_day'].values, rug=True, bins=24)

In [ ]:
plt.figure(figsize=(100,10))
plt.xticks(fontsize=70, rotation=0)
plt.yticks(fontsize=30)
plt.xlabel('Hour of the Day', fontsize=70)
sns.distplot(datadf['hour'], color='blue', label='Entries Without word',hist=False)
sns.distplot( datadf[datadf['found']==1]['hour'].values, color="red", label="Entries With Word",hist=False)
plt.legend(fontsize='x-large', title_fontsize='70')